## Imports

In [102]:
# DO NOT CHANGE THESE LINES.
using Suppressor
@suppress begin
    using DataFrames, CSV, Random, Statistics, Serialization, LazyJSON, StatsBase, DecisionTree, ScientificTypes, MLJ
end


## Paths

In [103]:
# DO NOT CHANGE THESE LINES 
ROOT_DIR = dirname(pwd())
MODEL_INPUTS_OUTPUTS = joinpath(ROOT_DIR, "model_inputs_outputs")
INPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "inputs")
INPUT_SCHEMA_DIR = joinpath(INPUT_DIR, "schema")
DATA_DIR = joinpath(INPUT_DIR, "data")
OUTPUT_DIR = joinpath(MODEL_INPUTS_OUTPUTS, "outputs")
TRAIN_DIR = joinpath(DATA_DIR, "training")
TEST_DIR = joinpath(DATA_DIR, "testing")
MODEL_PATH = joinpath(MODEL_INPUTS_OUTPUTS, "model")
MODEL_ARTIFACTS_PATH = joinpath(MODEL_PATH, "artifacts")
OHE_ENCODER_FILE = joinpath(MODEL_ARTIFACTS_PATH, "ohe.jld2")
PREDICTOR_DIR_PATH = joinpath(MODEL_ARTIFACTS_PATH, "predictor")
PREDICTOR_FILE_PATH = joinpath(PREDICTOR_DIR_PATH, "predictor.ser")
IMPUTATION_FILE = joinpath(MODEL_ARTIFACTS_PATH, "imputation.ser")
TOP_CATEGORIES = joinpath(MODEL_ARTIFACTS_PATH, "top_categories.ser")
PREDICTIONS_DIR = joinpath(OUTPUT_DIR, "predictions")
PREDICTIONS_FILE = joinpath(PREDICTIONS_DIR, "predictions.csv")
TARGET_LEVELS = joinpath(MODEL_ARTIFACTS_PATH, "target_levels.ser")



if !isdir(MODEL_ARTIFACTS_PATH)
    mkdir(MODEL_ARTIFACTS_PATH)
end
if !isdir(PREDICTOR_DIR_PATH)
    mkdir(PREDICTOR_DIR_PATH)
end

### Reading the schema

In [104]:
# Reading a schema from a JSON file and extracting features
file_name = first(filter(x -> endswith(x, "json"), readdir(INPUT_SCHEMA_DIR)))
schema_path = joinpath(INPUT_SCHEMA_DIR, file_name)
schema_string = read(schema_path, String)  # Read file content as a string
schema = LazyJSON.parse(schema_string)
features = schema["features"]

# Identifying numeric, categorical, and nullable features
numeric_features = String[]
categorical_features = String[]
nullable_features = String[]

for f in features
    if f["dataType"] == "CATEGORICAL"
        push!(categorical_features, f["name"])
    else
        push!(numeric_features, f["name"])
    end
    if f["nullable"]
        push!(nullable_features, f["name"])
    end
end

# Extracting ID and target features
id_feature = schema["id"]["name"]
target_feature = schema["target"]["name"]
target_classes = schema["target"]["classes"]

3-element LazyJSON.Array{Nothing, String}:
 "EI"
 "IE"
 "N"

### Reading test data.

In [105]:
file_name = filter(x -> occursin(".csv", x), readdir(TEST_DIR))[1]
file_path = joinpath(TEST_DIR, file_name)
df = DataFrame(CSV.File(file_path))

,Instance,Position_-30,Position_-29,Position_-28,Position_-27
,String31,String1,String1,String1,String1
1,HUMMHCW1B-ACCEPTOR-1398,G,G,A,C
2,HUMIL1B-DONOR-3748,G,C,A,C
3,HUMACCYBA-ACCEPTOR-2599,A,G,C,T
4,HUMRGNTSA-NEG-2161,C,T,G,T
5,HUMMHCW1B-DONOR-3405,G,C,T,C
6,HUMGCB1-ACCEPTOR-4081,C,C,G,G
7,HUM3ALPH-NEG-1,G,A,A,T
8,HUMA1GP01-NEG-301,G,C,G,G
9,HUMMYC3L-DONOR-402,C,C,G,C


## Data preprocessing
Note that when we work with testing data, we have to impute using the same values learned during training. This is to avoid data leakage.

In [106]:
imputation_values = open(deserialize, IMPUTATION_FILE)
for column in nullable_features
    df[!, Symbol(column)] .= coalesce.(df[!, Symbol(column)], get(imputation_values, string(column), missing))
end

# Saving the id column in a different variable
ids = df[!, Symbol(id_feature)]

# Dropping the id from the DataFrame
select!(df, Not([Symbol(id_feature)]))

,Position_-30,Position_-29,Position_-28,Position_-27,Position_-26,Position_-25
,String1,String1,String1,String1,String1,String1
1,G,G,A,C,G,G
2,G,C,A,C,C,T
3,A,G,C,T,A,A
4,C,T,G,T,C,T
5,G,C,T,C,T,G
6,C,C,G,G,A,T
7,G,A,A,T,T,C
8,G,C,G,G,G,A
9,C,C,G,C,T,C


### Encoding
We encode the data using the same encoder that we saved during training.

In [107]:
loaded_top_categories = open(deserialize, TOP_CATEGORIES)

# Function to one-hot encode only the top 3 categories
function one_hot_top_categories!(df, top_categories)
    for (feature, top_cats) in top_categories
        if length(top_cats) == 2  # Handle the binary case
            # Assuming the first category in top_cats is treated as 'true'
            new_col_name = "$(feature)_binary"
            df[!, new_col_name] = df[!, feature] .== top_cats[1]
        else  # Handle the general case
            for cat in top_cats
                new_col_name = "$(feature)_$(cat)"
                df[!, new_col_name] = df[!, feature] .== cat
            end
        end
        select!(df, Not(Symbol(feature)))  # Drop the original feature column
    end
end



one_hot_top_categories!(df, loaded_top_categories)

### Making predictions & Creating Predictions DataFrame
Using the model saved during training.

In [108]:
model = open(deserialize, PREDICTOR_FILE_PATH)
predictions = apply_forest_proba(model, Matrix(df), String.(target_classes))
predictions = DataFrame(predictions, Symbol.(target_classes))
insertcols!(predictions, 1, id_feature => ids)
CSV.write(PREDICTIONS_FILE, predictions)

"/Users/moo/Desktop/Upwork/rt-ML/Regression/Julia/Julia-Random-Forest-Classifier-Template/model_inputs_outputs/outputs/predictions/predictions.csv"